In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.regression import LabeledPoint
from sklearn import metrics
from numpy import array
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
sc = SparkContext(appName="ClassificationOfWineDataset")

In [2]:
#getting data
data = sc.textFile("D://NEU - Big Data and Intelligent Analytics/Assignment 1/winequality-white-classification-parsed format.csv")

In [3]:
def line_to_field(line):
    #space_separated_line = line.replace(';', ' ')
    string_array = line.split(',')
    #float_array = map(float, string_array)
    #float_array[11] = np.where(float_array[10] > 5.0, 1, 0)
    #float_array[11]=(float_array[10] > 5) 
    return LabeledPoint(string_array[0], string_array[1].split(' '))


In [4]:
parsedData = data.map(line_to_field)

In [5]:
#Spliting data into training and test data
(training, test) = parsedData.randomSplit([0.7, 0.3])

In [16]:
# Build the model
model = LogisticRegressionWithLBFGS.train(training, iterations=200,regParam= 0.1)

In [17]:
# Evaluating the model on training data
labelsAndPreds = training.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_training = labelsAndPreds.filter(lambda (v, p): v == p).count() / float(training.count())
print("Accuracy(training) = " + str(accuracy_training))
x = np.array(labelsAndPreds.collect())

Accuracy(training) = 0.790935672515


In [18]:
# Evaluating the model on test data
labelsAndPreds1 = test.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_test = labelsAndPreds1.filter(lambda (v, p): v == p).count() / float(test.count())
print("Accuracy(test) = " + str(accuracy_test))
y = np.array(labelsAndPreds1.collect())

Accuracy(test) = 0.791610284168


In [19]:
#clearing the threshold for getting scores as prediction.
model.clearThreshold()

In [20]:
#making an array of true lables and its predicted scores for training data
scoreandlabels = np.array(training.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Training Area under curve  = " + str(roc_auc_score(scoreandlabels[:,0],scoreandlabels[:,1]))
print "Training Area under Precision-Recall = " + str(average_precision_score(scoreandlabels[:,0],scoreandlabels[:,1]))

Training Area under curve  = 0.778800235262
Training Area under Precision-Recall = 0.506326363269


In [21]:
#making an array of true lables and its predicted scores for test data
scoreandlabels1 = np.array(test.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Test Area under curve  = " + str(roc_auc_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))
print "Test Area under Precision-Recall = " + str(average_precision_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))

Test Area under curve  = 0.771498264378
Test Area under Precision-Recall = 0.50243791689


In [22]:
#Making confusion matrix for training data
confusion_matrix(x[:,0],x[:,1])

array([[2673,   10],
       [ 705,   32]])

In [23]:
#Making confusion matrix for test data
confusion_matrix(y[:,0],y[:,1])

array([[1152,    3],
       [ 305,   18]])

In [24]:
#checking the weights for the various features
model.weights

DenseVector([-0.0258, -1.2606, -0.1319, -0.0015, -10.0896, 0.0054, -0.0028, -6.7387, 0.5621, 0.4034, 0.4302])

In [220]:
sc.stop()